## Importa Bibliotecas

In [1]:
import pandas as pd
import requests
import io
import urllib.request
import json
import urllib3
from datetime import datetime
import os

# Metodologia para varredura de arquivos

Verfica o ano atual e monta uma lista dos três últimos anos.

In [2]:
year = datetime.now().year
years = [year, year-1, year-2]
years

[2021, 2020, 2019]

Verifica se existem arquivos disponíveis para os últimos três anos. Caso haja arquivos os mesmos são salvos em arquivo local em '.csv'. Caso não haja arquivos disponíveis o mesmo aponta uma mensagem de erro. Ademais é realizada a renomeação das colunas do arquivo disponível.

In [3]:
rename_columns = {'CO_ANO': 'ANO',
                  'CO_MES': 'MES',
                  'CO_NCM': 'COD_NCM',
                  'CO_UNID': 'COD_UNIDADE',
                  'CO_PAIS': 'COD_PAIS',
                  'SG_UF_NCM': 'SG_UF',
                  'CO_VIA': 'COD_VIA',
                  'CO_URF': 'COD_URF',
                  'QT_ESTAT': 'VL_QUANTIDADE',
                  'KG_LIQUIDO': 'VL_PESO_KG',
                 }

In [4]:
def get_data(years, type_mov='EXP'):
    for year in years:
        url = f'https://balanca.economia.gov.br/balanca/bd/comexstat-bd/ncm/{type_mov}_{year}.csv'
        urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

        data=requests.get(url, verify=False).content
        try:
            df=pd.read_csv(io.StringIO(data.decode('utf-8')), sep=';')
            df.rename(columns=rename_columns, inplace=True)
            df.to_csv(f'{type_mov}_{year}.csv')
            print(f'Sucesso: {type_mov}_{year}')
        except Exception as e:
            print("Servidor indisponível. Erro:", e)

# Exportações

In [5]:
get_data(years, type_mov='EXP')

Sucesso: EXP_2021
Sucesso: EXP_2020
Sucesso: EXP_2019


# Importações

In [6]:
get_data(years, type_mov='IMP')

Sucesso: IMP_2021
Sucesso: IMP_2020
Sucesso: IMP_2019


# Junção de Arquivos

Inicialmente, carrega-se os dados de dada movimentação dos últimos três anos.

In [7]:
def load_files(years, type_mov='EXP'):
    year_now = pd.read_csv(f'{type_mov}_{years[0]}.csv')
    year_now.drop(columns='Unnamed: 0', inplace=True)
    year_old = pd.read_csv(f'{type_mov}_{years[1]}.csv')
    year_old.drop(columns='Unnamed: 0', inplace=True)
    year_more_old = pd.read_csv(f'{type_mov}_{years[2]}.csv')
    year_more_old.drop(columns='Unnamed: 0', inplace=True)
    
    return year_now, year_old, year_more_old

Inicialmente, será realizada a junção dos arquivos de exportação.

In [8]:
exp_year_now, exp_year_old, exp_year_more_old = load_files(years, type_mov='EXP')

Após carregadas as bases de dados pode-se fazer a junção das mesmas. O que resulta no dataframe a seguir.

In [9]:
result = exp_year_now.append(exp_year_old)
result = result.append(exp_year_more_old)
result

,ANO,MES,COD_NCM,COD_UNIDADE,COD_PAIS,SG_UF,COD_VIA,COD_URF,VL_QUANTIDADE,VL_PESO_KG,VL_FOB
0,2021,3,7032090,10,607,SP,1,817800,38,63,190
1,2021,2,85122011,11,158,SP,1,817800,85,132,6847
2,2021,5,84818099,11,97,SC,7,147600,6179,624,6248
3,2021,3,38091090,10,586,SC,7,917500,3300,3300,6370
4,2021,1,85045000,11,756,SP,1,817800,2,0,22
...,...,...,...,...,...,...,...,...,...,...,...
1393169,2019,10,65050019,10,589,SP,4,817700,300,300,861
1393170,2019,10,20059900,10,815,RS,1,927800,40210,40210,29975
1393171,2019,10,40093100,10,161,SP,4,817700,0,0,2
1393172,2019,10,7108000,10,573,ES,1,727600,13,13,46


Ajusta-se os dados do datraframe conforme requisitado: 'Acrescentando o zero à esquerda para algumas colunas'. Bem, como identifica que esse dataframe possui a movimentação de exportação.

In [10]:
result['COD_NCM'] = result['COD_NCM'].apply(lambda x: '0'+str(x))
result['COD_URF'] = result['COD_URF'].apply(lambda x: '0'+str(x))
result['COD_PAIS'] = result['COD_PAIS'].apply(lambda x: '0'+str(x))
result['MOVIMENTACAO'] = 'Exportação'
result

,ANO,MES,COD_NCM,COD_UNIDADE,COD_PAIS,SG_UF,COD_VIA,COD_URF,VL_QUANTIDADE,VL_PESO_KG,VL_FOB,MOVIMENTACAO
0,2021,3,07032090,10,0607,SP,1,0817800,38,63,190,Exportação
1,2021,2,085122011,11,0158,SP,1,0817800,85,132,6847,Exportação
2,2021,5,084818099,11,097,SC,7,0147600,6179,624,6248,Exportação
3,2021,3,038091090,10,0586,SC,7,0917500,3300,3300,6370,Exportação
4,2021,1,085045000,11,0756,SP,1,0817800,2,0,22,Exportação
...,...,...,...,...,...,...,...,...,...,...,...,...
1393169,2019,10,065050019,10,0589,SP,4,0817700,300,300,861,Exportação
1393170,2019,10,020059900,10,0815,RS,1,0927800,40210,40210,29975,Exportação
1393171,2019,10,040093100,10,0161,SP,4,0817700,0,0,2,Exportação
1393172,2019,10,07108000,10,0573,ES,1,0727600,13,13,46,Exportação


Carrega os dados de importação dos últimos três anos e faz a junção dos dados.

In [11]:
imp_year_now, imp_year_old, imp_year_more_old = load_files(years, 'IMP')

In [12]:
result_imp = imp_year_now.append(imp_year_old)
result_imp = result_imp.append(imp_year_more_old)
result_imp

,ANO,MES,COD_NCM,COD_UNIDADE,COD_PAIS,SG_UF,COD_VIA,COD_URF,VL_QUANTIDADE,VL_PESO_KG,VL_FOB
0,2021,4,90271000,11,23,SP,1,817800,11289,4638,959920
1,2021,3,39269090,10,160,AM,1,227600,87524,87524,632662
2,2021,2,84179000,10,232,RJ,4,817700,10,10,1554
3,2021,5,73209000,10,13,SP,1,817800,0,0,0
4,2021,3,40169990,10,149,RJ,4,717700,309,309,32253
...,...,...,...,...,...,...,...,...,...,...,...
1932750,2019,1,87168000,11,63,GO,1,817800,7,210,22501
1932751,2019,10,90321090,11,493,SC,1,817800,828,131,4075
1932752,2019,11,90138010,11,399,AM,4,227700,1,6,3181
1932753,2019,9,39231090,10,399,PR,1,917800,12,12,511


Renomeia colunas.

In [13]:
result_imp['COD_NCM'] = result_imp['COD_NCM'].apply(lambda x: '0'+str(x))
result_imp['COD_URF'] = result_imp['COD_URF'].apply(lambda x: '0'+str(x))
result_imp['COD_PAIS'] = result_imp['COD_PAIS'].apply(lambda x: '0'+str(x))
result_imp['MOVIMENTACAO'] = 'Importação'
result_imp

,ANO,MES,COD_NCM,COD_UNIDADE,COD_PAIS,SG_UF,COD_VIA,COD_URF,VL_QUANTIDADE,VL_PESO_KG,VL_FOB,MOVIMENTACAO
0,2021,4,090271000,11,023,SP,1,0817800,11289,4638,959920,Importação
1,2021,3,039269090,10,0160,AM,1,0227600,87524,87524,632662,Importação
2,2021,2,084179000,10,0232,RJ,4,0817700,10,10,1554,Importação
3,2021,5,073209000,10,013,SP,1,0817800,0,0,0,Importação
4,2021,3,040169990,10,0149,RJ,4,0717700,309,309,32253,Importação
...,...,...,...,...,...,...,...,...,...,...,...,...
1932750,2019,1,087168000,11,063,GO,1,0817800,7,210,22501,Importação
1932751,2019,10,090321090,11,0493,SC,1,0817800,828,131,4075,Importação
1932752,2019,11,090138010,11,0399,AM,4,0227700,1,6,3181,Importação
1932753,2019,9,039231090,10,0399,PR,1,0917800,12,12,511,Importação


Após formado os dataframes de importação e exportação é necessário unir esses dataframes.

In [14]:
comex = result.append(result_imp)
comex 

,ANO,MES,COD_NCM,COD_UNIDADE,COD_PAIS,SG_UF,COD_VIA,COD_URF,VL_QUANTIDADE,VL_PESO_KG,VL_FOB,MOVIMENTACAO
0,2021,3,07032090,10,0607,SP,1,0817800,38,63,190,Exportação
1,2021,2,085122011,11,0158,SP,1,0817800,85,132,6847,Exportação
2,2021,5,084818099,11,097,SC,7,0147600,6179,624,6248,Exportação
3,2021,3,038091090,10,0586,SC,7,0917500,3300,3300,6370,Exportação
4,2021,1,085045000,11,0756,SP,1,0817800,2,0,22,Exportação
...,...,...,...,...,...,...,...,...,...,...,...,...
1932750,2019,1,087168000,11,063,GO,1,0817800,7,210,22501,Importação
1932751,2019,10,090321090,11,0493,SC,1,0817800,828,131,4075,Importação
1932752,2019,11,090138010,11,0399,AM,4,0227700,1,6,3181,Importação
1932753,2019,9,039231090,10,0399,PR,1,0917800,12,12,511,Importação


# Salva o novo dataframe

In [15]:
comex.to_csv('f_comex.csv')

# Exclui arquivos

Alguns arquivos podem ser exlcuídos, pois apenas o arquivo 'f_comex.csv' é necessário.

In [16]:
for year in years:
    os.remove(f'EXP_{year}.csv')

In [17]:
for year in years:
    os.remove(f'IMP_{year}.csv')